### init

In [2]:
import os
import shutil
import subprocess as sp
import time

import numpy as np
from pymatgen.core.structure import Structure

cwd_dir = '/home/lonya/doc/dptb/YIG'
pseudo_dir = os.path.join(cwd_dir, 'pseudo')
orbital_dir = os.path.join(cwd_dir, 'orbital')
scf_dir = os.path.join(cwd_dir, 'scf')
nscf_dir = os.path.join(cwd_dir, 'nscf')

os.makedirs(scf_dir, exist_ok=True)
os.makedirs(nscf_dir, exist_ok=True)

## 1.generate data (DFT)
### scf
dft preparation (ABACUS)

In [13]:
np.random.seed(42)

os.chdir(cwd_dir)

structure = Structure.from_file('YIG.cif')
primitive = structure.get_primitive_structure()

work_dir = scf_dir
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)

cart_coords = primitive.cart_coords
numbers = primitive.atomic_numbers
NUM_ATOM = len(numbers)
lattice = primitive.lattice.matrix
frac_coords = primitive.frac_coords

Y_frac_coords_str = ''
FeO_frac_coords_str = ''
FeT_frac_coords_str = ''
O_frac_coords_str = ''

for i in range(NUM_ATOM):
    if numbers[i] == 39:
        Y_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2]:.16f} 0 0 0\n'
    elif numbers[i] == 26:
        xj = 0
        for j in range(3):
            xj = frac_coords[i, j]
            while xj > 0.24:
                xj -= 0.25
            if xj > 0.12:
                FeT_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2]:.16f} 0 0 0\n'
                break
        if xj < 0.12:
            FeO_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2]:.16f} 0 0 0\n'
    elif numbers[i] == 8:
        O_frac_coords_str += f'{frac_coords[i, 0]:.16f} {frac_coords[i, 1]:.16f} {frac_coords[i, 2]:.16f} 0 0 0\n'
    else:
        print(numbers[i])
        raise ValueError("numbers[i] error")

primitive.to('poscar', 'POSCAR')

in_file = fr"""ATOMIC_SPECIES
Y 88.906 Y_ONCV_PBE-1.0.upf
FeO 55.845 Fe_ONCV_PBE-1.0.upf
FeT 55.845 Fe_ONCV_PBE-1.0.upf
O 15.999 O_ONCV_PBE-1.0.upf

NUMERICAL_ORBITAL
Y_gga_8au_100Ry_4s2p2d1f.orb
Fe_gga_7au_100Ry_4s2p2d1f.orb
Fe_gga_7au_100Ry_4s2p2d1f.orb
O_gga_6au_100Ry_2s2p1d.orb

LATTICE_CONSTANT
1.8897259886 # 1.8897259886 Bohr = 1.0 Angstrom

LATTICE_VECTORS
{lattice[0, 0]:.16f} {lattice[0, 1]:.16f} {lattice[0, 2]:.16f}
{lattice[1, 0]:.16f} {lattice[1, 1]:.16f} {lattice[1, 2]:.16f}
{lattice[2, 0]:.16f} {lattice[2, 1]:.16f} {lattice[2, 2]:.16f}

ATOMIC_POSITIONS
Direct

Y
0.0
12
{Y_frac_coords_str}

FeO
-5.0
8
{FeO_frac_coords_str}

FeT
5.0
12
{FeT_frac_coords_str}

O
0.0
48
{O_frac_coords_str}
"""
with open('STRU', 'w') as f:
    f.write(in_file)

in_file = fr"""K_POINTS
0
Gamma
3 3 3 0 0 0
"""
with open('KPT', 'w') as f:
    f.write(in_file)

in_file = fr"""INPUT_PARAMETERS
pseudo_dir {pseudo_dir}
orbital_dir {orbital_dir}
ntype                   4
nspin                   2
calculation             scf
basis_type              lcao
ecutwfc                 100
scf_thr                 1.0e-6
scf_nmax                100
gamma_only              0
symmetry 0

smearing_method         gaussian
smearing_sigma          0.05

mixing_type             pulay
mixing_beta             0.4

dft_plus_u    0
orbital_corr    -1 2 2 -1
hubbard_u    0.0 2.0 2.0 0.0

out_chg 1
"""

with open('INPUT', 'w') as f:
    f.write(in_file)

dft calculation (scf)

In [14]:
os.chdir(scf_dir)
os.environ['OMP_NUM_THREADS'] = '8'

sp.run(['mpirun', '-np', '1', 'abacus'])

                                                                                     
                              ABACUS v3.6.0

               Atomic-orbital Based Ab-initio Computation at UStc                    

                     Website: http://abacus.ustc.edu.cn/                             
               Documentation: https://abacus.deepmodeling.com/                       
                  Repository: https://github.com/abacusmodeling/abacus-develop       
                              https://github.com/deepmodeling/abacus-develop         
                      Commit: 059fc16 (Sat Apr 13 12:08:33 2024 +0800)

 Wed May  8 09:26:22 2024
 MAKE THE DIR         : OUT.ABACUS/
 RUNNING WITH DEVICE  : CPU / 13th Gen Intel(R) Core(TM) i9-13900K

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 Pseudopotentials with additional electrons can yield (more) accurate outcomes, but may be less efficient.
 If you're confident th

CompletedProcess(args=['mpirun', '-np', '1', 'abacus'], returncode=0)

### nscf
dft preparation

In [10]:
os.chdir(nscf_dir)
os.makedirs(os.path.join(nscf_dir, 'OUT.ABACUS'), exist_ok=True)
shutil.copyfile(os.path.join(scf_dir, 'OUT.ABACUS/SPIN1_CHG.cube'), os.path.join(nscf_dir, 'OUT.ABACUS/SPIN1_CHG.cube'))
shutil.copyfile(os.path.join(scf_dir, 'OUT.ABACUS/SPIN2_CHG.cube'), os.path.join(nscf_dir, 'OUT.ABACUS/SPIN2_CHG.cube'))
shutil.copyfile(os.path.join(scf_dir, 'OUT.ABACUS/onsite.dm'), os.path.join(nscf_dir, 'OUT.ABACUS/onsite.dm'))
shutil.copyfile(os.path.join(scf_dir, 'OUT.ABACUS/istate.info'), os.path.join(nscf_dir, 'OUT.ABACUS/istate.info'))
shutil.copyfile(os.path.join(scf_dir, 'STRU'), os.path.join(nscf_dir, 'STRU'))

with open('STRU', 'w') as f:
    f.write(in_file)

in_file = fr"""K_POINTS
5
Line
0.0 0.5 0.0 20 // H
0.5 0.5 0.5 20 // 1 1 1
0.0 0.0 0.0 20 // G
0.0 0.0 0.5 20 // 0 0 1
0.25 0.25 0.25 1 // P
"""
with open('KPT', 'w') as f:
    f.write(in_file)
    
in_file = fr"""INPUT_PARAMETERS
pseudo_dir {pseudo_dir}
orbital_dir {orbital_dir}
ntype                   4
nspin                   2
calculation             nscf
basis_type              lcao
ecutwfc                 100
scf_thr                 1.0e-6
scf_nmax                100
gamma_only              0
symmetry 0

ks_solver    genelpa
smearing_method         gaussian
smearing_sigma          0.05

mixing_type             pulay
mixing_beta             0.4

dft_plus_u    0
orbital_corr    -1 2 2 -1
hubbard_u    0.0 5.0 5.0 0.0

init_chg file
out_band 1
"""

with open('INPUT', 'w') as f:
    f.write(in_file)

In [11]:
os.chdir(nscf_dir)
os.environ['OMP_NUM_THREADS'] = '8'

sp.run(['mpirun', '-np', '1', 'abacus'])

                                                                                     
                              ABACUS v3.6.0

               Atomic-orbital Based Ab-initio Computation at UStc                    

                     Website: http://abacus.ustc.edu.cn/                             
               Documentation: https://abacus.deepmodeling.com/                       
                  Repository: https://github.com/abacusmodeling/abacus-develop       
                              https://github.com/deepmodeling/abacus-develop         
                      Commit: 059fc16 (Sat Apr 13 12:08:33 2024 +0800)

 Wed May 15 11:25:15 2024
 MAKE THE DIR         : OUT.ABACUS/
 RUNNING WITH DEVICE  : CPU / 13th Gen Intel(R) Core(TM) i9-13900K

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 Pseudopotentials with additional electrons can yield (more) accurate outcomes, but may be less efficient.
 If you're confident th

CompletedProcess(args=['mpirun', '-np', '1', 'abacus'], returncode=0)